# Load data

In [1]:
import pandas as pd
import numpy as np

df_Train = pd.read_hdf('All_train.hdf', key='train')

In [2]:
df_Train.head()

,shop_id,item_id,date_block_num,target
22987,2,30,0,0.0
20994,2,31,0,0.0
20995,2,32,0,0.0
22492,2,33,0,1.0
22491,2,38,0,0.0


# Feature Engineering

In [4]:
# get national holidays from https://www.officeholidays.com/countries/russia/2013.php
num_holidays = [6, 1, 1, 0, 5, 1, 0, 0, 0, 0, 1, 0, 6, 1, 1, 0, 3, 2, 0, 0, 0, 0, 2, 0, 7, 1, 1, 0, 4, 4, 0, 0, 0, 0, 1, 0]
for i in range(len(num_holidays)):
    df_Train.loc[df_Train['date_block_num'] == i, 'num_holidays'] = num_holidays[i]

In [7]:
# item mean encoding using Expanding mean scheme
global_mean = df_Train.target.mean()

cumsum = df_Train.groupby('item_id')['target'].cumsum() - df_Train['target']
cumcount = df_Train.groupby('item_id')['target'].cumcount()

df_Train['item_target_enc']= cumsum / cumcount
df_Train['item_target_enc'].fillna(global_mean, inplace=True)

In [8]:
# shop mean encoding using Expanding mean scheme
global_mean = df_Train.target.mean()

cumsum = df_Train.groupby('shop_id')['target'].cumsum() - df_Train['target']
cumcount = df_Train.groupby('shop_id')['target'].cumcount()

df_Train['shop_target_enc']= cumsum / cumcount
df_Train['shop_target_enc'].fillna(global_mean, inplace=True)

In [26]:
df_Train['target'].clip(lower=-2,upper=20).value_counts(ascending=True)

-2.0           6
-1.0         246
 19.0        570
 18.0        628
 17.0        761
 16.0        836
 15.0       1003
 14.0       1161
 13.0       1482
 12.0       1714
 11.0       2082
 10.0       2810
 9.0        3553
 8.0        4972
 20.0       6541
 7.0        6600
 6.0        9838
 5.0       15026
 4.0       24321
 3.0       45773
 2.0      109854
 1.0      359469
 0.0     6683554
Name: target, dtype: int64

In [3]:
# KNN
import KNN
KNN_feats = KNN.train(df_Train.drop(columns=['target']).values, df_Train['target'].clip(lower=0,upper=20).values)
columns = []
for i in range(KNN_feats.shape[1]):
    columns.append('KNN' + str(i))
KNN_feats = pd.DataFrame(KNN_feats, columns=columns)
df_Train = pd.concat([df_Train, KNN_feats], axis=1)

minkowski


AssertionError: 